In [1]:
from pyscf import gto, scf, ao2mo, fci
import CC, Wick


In [2]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [3]:
cisolver.kernel()

(-107.64818429172374,
 array([[ 9.58760058e-01,  9.27989225e-17, -4.07840258e-17, ...,
         -8.21265318e-11, -3.56684428e-22, -4.53355275e-12],
        [ 9.27989225e-17, -2.51901862e-02, -7.61837710e-17, ...,
          5.93963724e-20, -2.06868201e-22,  7.38158660e-22],
        [-4.07840258e-17, -7.61837710e-17, -2.00131682e-02, ...,
         -1.56970194e-18, -1.15523541e-07, -1.60803198e-21],
        ...,
        [-8.21265318e-11,  5.93963724e-20, -1.56970194e-18, ...,
         -2.87548220e-06,  3.74800215e-19, -5.43701894e-08],
        [-3.56684428e-22, -2.06868201e-22, -1.15523541e-07, ...,
          3.74800215e-19, -9.00895631e-08, -8.19042296e-23],
        [-4.53355275e-12,  7.38158660e-22, -1.60803198e-21, ...,
         -5.43701894e-08, -8.19042296e-23, -2.35887156e-08]]))

In [4]:
H2CCSDClusterOperator = CC.coupledCluster(mf, range(1,3))

KeyboardInterrupt: 

In [4]:
print(H2CCSDClusterOperator)

-0.0
 + -4.394149808727142e-17 * a^{1\alpha}a_{0\alpha}
 + -4.394149808727142e-17 * a^{1\beta}a_{0\beta}
 + -0.06883332874447325 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.06883332874447325 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


In [5]:
waveOperator = CC.exponentialOperator(H2CCSDClusterOperator, 3)
hamiltonian = CC.get1bodyHamiltonian(mf) + CC.get2bodyHamiltonian(mf)
inverseWaveOperator = CC.exponentialOperator((-1) * H2CCSDClusterOperator, 3)

In [6]:
CCSDEnergy = Wick.vacuumExpectationValue(inverseWaveOperator * hamiltonian * waveOperator, [1,0]) + mol.energy_nuc()
print(CCSDEnergy)

-1.1284543355083052
